# EOF + PC demo (Binder-ready)

Synthetic example of EOF spatial patterns and PC time series
using Cartopy + Matplotlib.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cmocean
from matplotlib.gridspec import GridSpec

plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['DejaVu Sans'],
    'font.size': 8,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight'
})

In [ ]:
def plot_eof_summary(eof_modes, pcs, variance_explained,
                     lons, lats, time,
                     n_modes=3,
                     save_path='EOF_summary.pdf'):

    fig = plt.figure(figsize=(7.2, 9))
    gs = GridSpec(n_modes, 2, figure=fig,
                  width_ratios=[1.5, 1],
                  hspace=0.3,
                  wspace=0.3)

    proj = ccrs.PlateCarree(central_longitude=180)

    for i in range(n_modes):
        ax_map = fig.add_subplot(gs[i, 0], projection=proj)
        eof = eof_modes[i]

        levels = np.linspace(-np.abs(eof).max(),
                             np.abs(eof).max(), 21)

        cf = ax_map.contourf(
            lons, lats, eof,
            levels=levels,
            cmap=cmocean.cm.balance,
            transform=ccrs.PlateCarree(),
            extend='both'
        )

        ax_map.set_extent([120, 290, -60, 60],
                          crs=ccrs.PlateCarree())
        ax_map.coastlines(linewidth=0.4)
        ax_map.add_feature(cfeature.LAND, facecolor='lightgray')

        plt.colorbar(cf, ax=ax_map,
                     orientation='horizontal', pad=0.05)

        ax_map.set_title(
            f'EOF {i+1} ({variance_explained[i]:.1f}%)',
            fontsize=8, fontweight='bold'
        )

        ax_ts = fig.add_subplot(gs[i, 1])
        pc = pcs[i]
        pc_norm = (pc - pc.mean()) / pc.std()

        ax_ts.fill_between(time, 0, pc_norm,
                           where=(pc_norm >= 0),
                           color='#d73027', alpha=0.7)
        ax_ts.fill_between(time, 0, pc_norm,
                           where=(pc_norm < 0),
                           color='#4575b4', alpha=0.7)

        ax_ts.plot(time, pc_norm, 'k-', linewidth=0.4)
        ax_ts.axhline(0, color='k', linewidth=0.4, alpha=0.3)

        ax_ts.set_ylabel(f'PC{i+1}', fontsize=7)
        ax_ts.spines['top'].set_visible(False)
        ax_ts.spines['right'].set_visible(False)
        ax_ts.grid(True, alpha=0.2)

        if i == n_modes - 1:
            ax_ts.set_xlabel('Year', fontsize=7)

    plt.savefig(save_path)
    plt.close()

In [ ]:
lats = np.linspace(-60, 60, 61)
lons = np.linspace(120, 290, 86)
lon2d, lat2d = np.meshgrid(lons, lats)

eof1 = np.cos(np.deg2rad(lat2d)) * np.sin(np.deg2rad(lon2d - 180))
eof2 = np.sin(2*np.deg2rad(lat2d)) * np.cos(np.deg2rad(lon2d - 200))
eof3 = np.cos(np.deg2rad(lat2d/2)) * np.sin(2*np.deg2rad(lon2d - 160))

eof_modes = [eof1, eof2, eof3]

time = np.arange(1980, 2021)
pc1 = np.sin(2*np.pi*(time-1980)/6)  + 0.3*np.random.randn(len(time))
pc2 = np.sin(2*np.pi*(time-1980)/12) + 0.3*np.random.randn(len(time))
pc3 = np.sin(2*np.pi*(time-1980)/20) + 0.3*np.random.randn(len(time))

pcs = [pc1, pc2, pc3]
variance_explained = np.array([35.0, 18.0, 10.0])

In [ ]:
plot_eof_summary(
    eof_modes, pcs, variance_explained,
    lons, lats, time,
    n_modes=3,
    save_path='EOF_summary.pdf'
)

print('EOF_summary.pdf generated')